In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statistics import mean
import matplotlib.font_manager

from RoomMeetingAssignments import *
from PeerConnection import *
from dt_calc_models import *

from rmsops import *

fontName = 'YS Text'
fontColor = '#FF6427'
fontColor2 = '#000000'

font_files = matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype')
for font_file in font_files:
    try: 
        matplotlib.font_manager.fontManager.addfont(font_file)
    except Exception:
        print('failed to add font ' + font_file)
        #ignore
font = {'family' : 'YS Text',
        'size'   : 16}

matplotlib.rc('font', **font)

failed to add font /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf


In [2]:
#to speed up testing
MAX_NUM_ROWS=50000
# MAX_NUM_ROWS=10000000000

MEDIASERVER_IDLE_TIMEOUT = 60

#model parameters
ISLAND_SIZES = [50] * 5
RECONNECT_DOWNTIME_SECONDS = 10

#parameters to optimize
GRACE_PERIOD_SECONDS = 1800

#disruptions are done one island at a time. When one rolls out, other islands stay 100% available
DISRUPTION_BUDGET = 50